<a href="https://colab.research.google.com/github/RajkumarGalaxy/ComputerVision/blob/master/Avalanche_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Avalanche

A Python Library for Continual Learning

References:

Original Research Paper
https://arxiv.org/pdf/2104.00405v1.pdf

Github repository
https://github.com/ContinualAI/avalanche

Official Tutorial
https://github.com/ContinualAI/avalanche/tree/master/notebooks/from-zero-to-hero-tutorial

Official website
https://avalanche.continualai.org/



In [1]:
# Install Avalanche and its dependencies
!pip install git+https://github.com/ContinualAI/avalanche.git

  Cloning https://github.com/ContinualAI/avalanche.git to /tmp/pip-req-build-52zy56_7
  Running command git clone -q https://github.com/ContinualAI/avalanche.git /tmp/pip-req-build-52zy56_7
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 276kB 5.8MB/s 
     |████████████████████████████████| 2.1MB 37.1MB/s 
     |████████████████████████████████| 532kB 39.3MB/s 
     |████████████████████████████████| 163kB 37.8MB/s 
     |████████████████████████████████| 133kB 36.4MB/s 
     |████████████████████████████████| 102kB 9.4MB/s 
     |████████████████████████████████| 71kB 6.1MB/s 
  Created wheel for avalanche: filename=avalanche-0.0.1-cp37-none-any.whl size=300351 sha256=629a2a7a451d9cdd84d0dc83efe08d2226199d9ced7ef8baf82b8e05ba691525
  Stored in directory: /tmp/pip-ephem-wheel-cache-2715ozjv/wheels/45/3e/b7/1d78eeb6c42557cc5b6d727b2fed5b8ed3895bbcd23d9b6d29
Successfully

In [5]:
import torch
# use CrossEntropyLoss
from torch.nn import CrossEntropyLoss
# use stochastic GD optimizer
from torch.optim import SGD
# import the PermutedMNIST dataset
from avalanche.benchmarks.classic import PermutedMNIST
# import the SimpleMLP dataset
from avalanche.models import SimpleMLP
# we will use Naive training strategy
from avalanche.training.strategies import Naive

In [6]:
# Configure the device settings
# check for CUDA GPU
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# model instantiation
model = SimpleMLP(num_classes = 10)

In [7]:
# Benchmark instantiation
perm_mnist = PermutedMNIST(n_experiences=3)
# split stream into train and test set
train_stream = perm_mnist.train_stream
test_stream = perm_mnist.test_stream

Failed to download (trying next):
HTTP Error 503: Service Unavailable




Extracting /root/.avalanche/data/mnist/MNIST/raw/train-images-idx3-ubyte.gz to /root/.avalanche/data/mnist/MNIST/raw




Extracting /root/.avalanche/data/mnist/MNIST/raw/train-labels-idx1-ubyte.gz to /root/.avalanche/data/mnist/MNIST/raw




Extracting /root/.avalanche/data/mnist/MNIST/raw/t10k-images-idx3-ubyte.gz to /root/.avalanche/data/mnist/MNIST/raw




Extracting /root/.avalanche/data/mnist/MNIST/raw/t10k-labels-idx1-ubyte.gz to /root/.avalanche/data/mnist/MNIST/raw

Processing...
Done!


/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:502: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:143.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [8]:
# define optimizer and loss function for training
optimizer = SGD(model.parameters(), lr=0.001, momentum=0.9)
criterion = CrossEntropyLoss()

In [9]:
# Instantiate Continual learning strategy
strategy = Naive(
    model, optimizer, criterion, train_mb_size=32, train_epochs=2,
    eval_mb_size=32, device=device)

In [10]:
results = []
# train and test epochs
for train_task in train_stream:
    # training
    strategy.train(train_task, num_workers=4)
    # evaluate the model and store the results
    results.append(strategy.eval(test_stream))

-- >> Start of training phase << --
-- Starting training on experience 0 (Task 0) from train stream --
0it [00:00, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


100%|██████████| 1875/1875 [00:40<00:00, 45.94it/s]
Epoch 0 ended.
	Loss_Epoch/train_phase/train_stream/Task000 = 0.4867
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.8612
100%|██████████| 1875/1875 [00:40<00:00, 45.96it/s]
Epoch 1 ended.
	Loss_Epoch/train_phase/train_stream/Task000 = 0.2556
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.9254
-- >> End of training phase << --
-- >> Start of eval phase << --
-- Starting eval on experience 0 (Task 0) from test stream --
100%|██████████| 313/313 [00:06<00:00, 50.59it/s]
> Eval on experience 0 (Task 0) from test stream ended.
	Loss_Exp/eval_phase/test_stream/Task000/Exp000 = 0.1826
	Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp000 = 0.9473
-- Starting eval on experience 1 (Task 1) from test stream --
100%|██████████| 313/313 [00:06<00:00, 51.03it/s]
> Eval on experience 1 (Task 1) from test stream ended.
	Loss_Exp/eval_phase/test_stream/Task001/Exp001 = 2.6939
	Top1_Acc_Exp/eval_phase/test_stream/Task001/Exp001 = 0.0759
-- S

In [11]:
results

[{'Loss_Epoch/train_phase/train_stream/Task000': 0.2555898996353149,
  'Loss_Exp/eval_phase/test_stream/Task000/Exp000': 0.1826079943150282,
  'Loss_Exp/eval_phase/test_stream/Task001/Exp001': 2.693893172073364,
  'Loss_Exp/eval_phase/test_stream/Task002/Exp002': 2.475934606552124,
  'Loss_Stream/eval_phase/test_stream': 1.7841452576468388,
  'Top1_Acc_Epoch/train_phase/train_stream/Task000': 0.9253666666666667,
  'Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp000': 0.9473,
  'Top1_Acc_Exp/eval_phase/test_stream/Task001/Exp001': 0.0759,
  'Top1_Acc_Exp/eval_phase/test_stream/Task002/Exp002': 0.1464,
  'Top1_Acc_Stream/eval_phase/test_stream': 0.38986666666666664},
 {'Loss_Epoch/train_phase/train_stream/Task000': 0.2555898996353149,
  'Loss_Epoch/train_phase/train_stream/Task001': 0.2216990418612957,
  'Loss_Exp/eval_phase/test_stream/Task000/Exp000': 0.20704203982353211,
  'Loss_Exp/eval_phase/test_stream/Task001/Exp001': 0.1539949437007308,
  'Loss_Exp/eval_phase/test_stream/Task002/

Thank you for your time!